In [25]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt

### 1 - Getting Games Data 


In [26]:
url = "https://stats.nba.com/stats/teamgamelogs?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlusMinus=N&Rank=N&Season=2021-22&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&VsConference=&VsDivision="

In [27]:
# using the headers to fool Adam Silver
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [28]:
response = requests.get(url=url, headers=headers).json()

In [29]:
data = response['resultSets'][0]['rowSet']

In [30]:
columns = response['resultSets'][0]['headers']

 ### 2 - Cleaning Data and Getting Points for Opponent

In [31]:
nba_df = pd.DataFrame(data, columns=columns)

In [32]:
nba_df.columns

Index(['SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK',
       'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
       'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
       'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
       'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK'],
      dtype='object')

In [33]:
nba_df['PLUS_MINUS']

0      -29.0
1       -8.0
2       14.0
3       -5.0
4        5.0
        ... 
2455    12.0
2456    23.0
2457     7.0
2458   -23.0
2459    -7.0
Name: PLUS_MINUS, Length: 2460, dtype: float64

In [34]:
nba_df['ABS_PLUS_MINUS'] = nba_df['PLUS_MINUS'].abs()

In [35]:
nba_df['ABS_PLUS_MINUS']

0       29.0
1        8.0
2       14.0
3        5.0
4        5.0
        ... 
2455    12.0
2456    23.0
2457     7.0
2458    23.0
2459     7.0
Name: ABS_PLUS_MINUS, Length: 2460, dtype: float64

In [36]:
tdf_w = nba_df[nba_df['WL'] == 'W']

In [37]:
tdf_L = nba_df[nba_df['WL'] == 'L']

In [38]:
nba_df['OPT_PTS'] = tdf_w['PTS'] - tdf_w['PLUS_MINUS']

In [39]:
nba_df['OPT_PTS'].update(tdf_L['PTS'] + tdf_L['ABS_PLUS_MINUS'])

In [40]:
nba_df.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,ABS_PLUS_MINUS,OPT_PTS
0,2021-22,1610612763,MEM,Memphis Grizzlies,0022101223,2022-04-10T00:00:00,MEM vs. BOS,L,48.0,39,...,301,240,532,179,376,994,1226,2368,29.0,139.0
1,2021-22,1610612754,IND,Indiana Pacers,0022101216,2022-04-10T00:00:00,IND @ BKN,L,48.0,47,...,38,12,2283,1637,1892,1918,263,1679,8.0,134.0
2,2021-22,1610612753,ORL,Orlando Magic,0022101227,2022-04-10T00:00:00,ORL vs. MIA,W,48.0,45,...,301,2166,1631,831,376,2324,293,402,14.0,111.0
3,2021-22,1610612743,DEN,Denver Nuggets,0022101220,2022-04-10T00:00:00,DEN vs. LAL,L,53.0,49,...,1226,380,323,1637,2458,274,21,1471,5.0,146.0
4,2021-22,1610612747,LAL,Los Angeles Lakers,0022101220,2022-04-10T00:00:00,LAL @ DEN,W,53.0,44,...,949,2166,532,1930,2049,1,8,922,5.0,141.0


In [41]:
mavs = nba_df[nba_df['TEAM_NAME'] == 'Dallas Mavericks']
suns = nba_df[nba_df['TEAM_NAME'] == 'Phoenix Suns']

In [45]:
mavs_mean_pts = mavs['PTS'].mean()
mavs_std_pts = mavs['PTS'].std()
mavs_mean_opts = mavs['OPT_PTS'].mean()
mavs_std_opts = mavs['OPT_PTS'].std()

suns_mean_pts = suns['PTS'].mean()
suns_std_pts = suns['PTS'].std()
suns_mean_opts = suns['OPT_PTS'].mean()
suns_std_opts = suns['OPT_PTS'].std()

In [49]:
def simulator():
    mavs_score = (np.random.normal(mavs_mean_pts, mavs_std_pts)+ np.random.normal(suns_mean_opts, suns_std_opts))/2
    suns_score = (np.random.normal(suns_mean_pts, suns_std_pts)+ np.random.normal(mavs_mean_opts, mavs_std_opts))/2
    if int(round(mavs_score)) > int(round(suns_score)):
        return 1
    elif int(round(mavs_score)) < int(round(suns_score)):
        return -1
    else: return 0

In [50]:
def nsimulator(n):
    gamesout = []
    team1win = 0
    team2win = 0
    tie = 0
    for i in range(n):
        gm = simulator()
        gamesout.append(gm)
        if gm == 1:
            team1win +=1 
        elif gm == -1:
            team2win +=1
        else: tie +=1 
    print('Dallas Win ', team1win/(team1win+team2win+tie),'%')
    print('Suns Win ', team2win/(team1win+team2win+tie),'%')
    print('Tie ', tie/(team1win+team2win+tie), '%')
    return gamesout

In [51]:
nsimulator(1000000)

Dallas Win  0.409533 %
Suns Win  0.556375 %
Tie  0.034092 %


[1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 0,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 0,
 -1,
 -1,
 0,
 0,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 0,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1